## Trial of Time Series Analysis by Tensorflow Probability - Autoregressive Model - Continued
#### Based on https://bayesiancomputationbook.com/markdown/chp_06.html#autoregressive-models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

In [4]:
# %load 'co2_by_month_data.py'
import numpy as np
import pandas as pd

def co2_by_month_data():

    co2_by_month = pd.read_csv("../data/monthly_mauna_loa_co2_20220721.csv", 
                               comment='#')
    co2_by_month['year_string'] = co2_by_month['year'].astype(str)
    co2_by_month['month_string'] = co2_by_month['month'].astype(str)
    co2_by_month['date_month'] = pd.to_datetime(co2_by_month['year_string'] 
                                        + '/' + co2_by_month['month_string'])
    co2_by_month.set_index('date_month', drop=True, inplace=True)
    co2_by_month.drop(columns=['year', 'month', 'year_string', 
                               'month_string'], inplace=True)
    co2_by_month['CO2'] = co2_by_month['average'].astype(np.float32)
    co2_by_month.drop(columns=['decimal date', 'average', 'deseasonalized', 
                               'ndays', 'sdev', 'unc'], inplace=True)

    num_forecast_steps = 12 * 10 # Forecast the final ten years
    co2_by_month_training_data = co2_by_month[:-num_forecast_steps]
    co2_by_month_testing_data = co2_by_month[-num_forecast_steps:]
    
    num_forecast_steps = 12 * 10 # Forecast the final ten years
    trend_all = np.linspace(0., 1., len(co2_by_month))[..., None]
    trend_all = trend_all.astype(np.float32)
    seasonality_all = pd.get_dummies(
       co2_by_month.index.month).values.astype(np.float32)
    trend = trend_all[:-num_forecast_steps, :]
    seasonality = seasonality_all[:-num_forecast_steps, :]
    
    return (co2_by_month, co2_by_month_training_data, 
            co2_by_month_testing_data, trend_all, seasonality_all, 
            trend, seasonality)

In [5]:
(co2_by_month, co2_by_month_training_data, 
co2_by_month_testing_data, trend_all, seasonality_all, 
trend, seasonality) = co2_by_month_data()

### (S)AR(I)MA(X)

#### Since birth rate is not available in easy to use way (The reference stores the data in R language data format), I continue to use CO2 data.

In [6]:
p, d, q = (1, 1, 1)

In [7]:
seasonal_p, seasonal_d, seasonal_q, period = (1, 1, 1, 12)

In [8]:
observed = co2_by_month.values

In [9]:
# Integrated to seasonal order
for _ in range(seasonal_d):
    observed = observed[period:] - observed[:-period]

In [10]:
# Integrated to order d
observed = tf.constant(np.diff(observed, n=d), tf.float32)

2022-08-12 17:39:01.344199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
r = max(p, q, seasonal_p * period, seasonal_q * period)

In [16]:
n_tp = seasonality_all.shape[0]

In [17]:
t = np.linspace(0, 1, n_tp, dtype=np.float32)

In [19]:
num_steps = 100

In [20]:
x = np.linspace(0, 50, num_steps)

In [27]:
from sarima_priors import generate_sarima_priors

In [28]:
sarima_priors = generate_sarima_priors(p, q, seasonal_p, seasonal_q)

In [14]:
from sarima_likelihood import generate_sarima_likelihood

In [18]:
sarima_likelihood = generate_sarima_likelihood(observed, r, t)

In [32]:
from run_mcmc_simple import run_mcmc_simple

In [33]:
%%time
target_log_prob_fn = lambda *x: sarima_priors.log_prob(*x) + sarima_likelihood(*x)

mcmc_samples, sampler_stats = run_mcmc_simple(
    1000, sarima_priors, n_chains=4, num_adaptation_steps=1000,
    target_log_prob_fn=target_log_prob_fn,
    seed=tf.constant([623453, 456345], dtype=tf.int32),
)

ValueError: Shape must be rank 1 but is rank 2
	 for 1th input and equation: ...,j->j... for '{{node mcmc_sample_chain/dual_averaging_step_size_adaptation___init__/_bootstrap_results/transformed_kernel_bootstrap_results/NoUTurnSampler/.bootstrap_results/process_args/maybe_call_fn_and_grads/value_and_gradients/value_and_gradient/einsum/Einsum}} = Einsum[N=2, T=DT_FLOAT, equation="...,j->j..."](mcmc_sample_chain/dual_averaging_step_size_adaptation___init__/_bootstrap_results/transformed_kernel_bootstrap_results/NoUTurnSampler/.bootstrap_results/process_args/maybe_call_fn_and_grads/value_and_gradients/value_and_gradient/ones_like, mcmc_sample_chain/dual_averaging_step_size_adaptation___init__/_bootstrap_results/transformed_kernel_bootstrap_results/NoUTurnSampler/.bootstrap_results/process_args/maybe_call_fn_and_grads/value_and_gradients/value_and_gradient/einsum/Einsum/inputs_1)' with input shapes: [4], [760,0].

In [34]:
run_mcmc = tf.function(tfp.experimental.mcmc.windowed_adaptive_nuts, autograph=False, jit_compile=True)